## DEMO VER 1

시간별로 잘린 문장을 -> T5 모델로 요약을 한후 -> 중요 문장과 주요 단어를 추출하여 노션으로 정리한다.


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### 데이터 전처리

'2023년'으로 시작하는 말이 있다면 제거한다.

### Segmentation by time

In [3]:
import pandas as pd
import re
import datetime

d = {
    'timestamp': [],
    'user': [],
    'chat': [],
}
def is_unnecessary_chat(line: str) -> bool:
    """실제 채팅 내용이 아닌 것들 확인"""
    if "님이 나갔습니다." in line:
        return True
    elif "님이 들어왔습니다." in line:
        return True
    elif "채팅방 관리자가 메시지를 가렸습니다." in line:
        return True
    elif "사진" == line:
        return True
    elif "이모티콘" == line:
        return True
    else:
        return False

In [4]:
with open("/content/drive/MyDrive/23-1 데캡디/데이터/데이터.txt", "r", encoding="utf-8") as f:
    content = f.read()
    lines = content.split("2023년 ")  # 한 대화에 \n이 여러번 들어간 것들이 있어서 일단 요걸로 split
    # 텍스트 파일의 맨 윗부분, 대화 내용에 2023년이 들어간 경우 삭제함.
    #pattern = r"^(오전|오후) \d{1,2}:\d{2}$"

    for line in lines[1:]:
        if line == '':
            continue
        elif is_unnecessary_chat(line):
            continue

        try:
            timestamp, user_chat = line.split(", ", 1)
            user, chat = user_chat.split(" : ", 1)
        except ValueError as e:
            print(e, line)
            continue

        timestamp = "2023년 " + timestamp
        d['timestamp'].append(timestamp)
        d['user'].append(user)
        d['chat'].append(chat[:-1])


df = pd.DataFrame(data=d)

df['timestamp'] = df['timestamp'].str.replace('오전', 'AM')
df['timestamp'] = df['timestamp'].str.replace('오후', 'PM')

df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y년 %m월 %d일 %p %I:%M")
df['prev_timestamp'] = df['timestamp'].shift(1)
df['timedelta'] = df['timestamp']-df['prev_timestamp']

thld_5min = []
thld_10min = []
thld_15min = []
thld_20min = []

# 클러스터링 기준 번호. 0부터 시작
thld_5min_cnt = 0
thld_10min_cnt = 0
thld_15min_cnt = 0
thld_20min_cnt = 0

for i, row in df[['timedelta']].iterrows():
    if row['timedelta'] > datetime.timedelta(minutes=5):
        thld_5min_cnt += 1
    if row['timedelta'] > datetime.timedelta(minutes=10):
        thld_10min_cnt += 1
    if row['timedelta'] > datetime.timedelta(minutes=15):
        thld_15min_cnt += 1
    if row['timedelta'] > datetime.timedelta(minutes=20):
        thld_20min_cnt += 1

    thld_5min.append(thld_5min_cnt)
    thld_10min.append(thld_10min_cnt)
    thld_15min.append(thld_15min_cnt)
    thld_20min.append(thld_20min_cnt)

df['thld_5min'] = thld_5min
df['thld_10min'] = thld_10min
df['thld_15min'] = thld_15min
df['thld_20min'] = thld_20min

print(df)
#df.to_excel("/content/drive/MyDrive/23-1 데캡디/데이터result.xlsx")

not enough values to unpack (expected 2, got 1) 1월 4일 오후 8:35

not enough values to unpack (expected 2, got 1) 1월 4일 오후 10:39, 송준용 | 디지털 마케팅 | 대표님이 부방장이 되었습니다.

not enough values to unpack (expected 2, got 1) 1월 5일 오전 12:00

not enough values to unpack (expected 2, got 1) 1월 6일 오전 12:09

not enough values to unpack (expected 2, got 1) 1월 7일 오전 12:02

not enough values to unpack (expected 2, got 1) 1월 8일 오전 12:24

not enough values to unpack (expected 2, got 1) 1월 9일 오전 12:07

not enough values to unpack (expected 2, got 1) 1월 10일 오전 12:13

not enough values to unpack (expected 2, got 1) 1월 11일 오전 12:08

not enough values to unpack (expected 2, got 1) 1월 12일 오전 12:58

not enough values to unpack (expected 2, got 1) 1월 13일 오전 12:01

not enough values to unpack (expected 2, got 1) 1월 14일 오전 12:20

not enough values to unpack (expected 2, got 1) 1월 15일 오전 1:35

not enough values to unpack (expected 2, got 1) 1월 16일 오전 12:37

not enough values to unpack (expected 2, got 1) 1월 17일 오전 12:08



In [5]:
df_grouped = df.groupby('thld_20min').agg({'chat': ' '.join})
df = df_grouped

### Summarization by T5

In [7]:
!pip install -q wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to('cuda')

In [75]:
summary_data = pd.DataFrame(columns=['text', 'summary'])
summary_data['text'] = df['chat']

In [77]:
count = 0

for index, row in df.iterrows():
    count += 1
    text = row['chat']
    
    inputs = ["summarize: " + text]

    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to('cuda')
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=256)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
    
    print(predicted_title)
    # 'summary'열 내용 채우기
    summary_data.loc[index, 'summary'] = predicted_title
    print(summary_data['summary'].count())
    if count > 10:
       break

# 새로운 DataFrame을 CSV 파일로 저장
summary_data.to_csv('/content/drive/MyDrive/chat_test.csv', index=False)

OpenAI와 microsoft가 미국에서 고소 당한 상태로 소송이 진행 중인데 캐나다에서는 현저한 노력을 해야 저작권을 인정해 주기 때문에 AI가 만들어낸 것들은 누구도 저작권이 인정되기 어려운 상태라고 한다.
1
ChatGPT가 놀라운 능력을 보이는 것 중 하나가 데이터의 포맷을 바꾸는 능력인데 드라마 스크립트를 넣어주면서 등장 인물과 그 사람들 간의 관계에 대한 그래프를 GraphViz 포맷으로
2
ChatGPT 4.0가 출시 5일만에 100만명의 사용자를 달성했다고 하더니 기업이나 연구소들이 가지고 있는 비공개 데이터들을 별도의 DB로 학습시켜 모델을 병합시킬 수 있게 상품화
3
한글로 질문하면 자체적으로 질문을 영어로 번역해서 답변을 생성하는 방식인데 답변을 생성할 때 사용하는 모델은 각각의 언어로 학습된 개별 모델이다.
4
참여 신청하는 곳이 있을까요?
5
한글로 질문을 입력하면 답변을 다 하지 않고 끊길 때가 있는데 영어로 대답해 달라고 하면 쭉 말하더라구요 한글로뿐만 아니라 다른 언어들도 끊겨서 답을 할 때가 있었다.
6
언어모델 중 가장 강력하며 비싼 다빈치 모델이 0.0200달러 / 1K 토큰이며 이는 4000자를 의미한다.
7
ai 글쓰기 관심이 있어서 들어왔는데 ai 글쓰기 관심이 있어서 들어왔습니다 ai 글쓰기 관심이 있어서 들어왔습니다 ai 글쓰기 관심이 있어서 들어왔습니다 ai 글쓰기 엑셀과 호환한 사례 입니다.
8
서울대학교 산업공학과에 다니고 있는 학생은 AI쪽 관심이 있어 공부시작했고 graph쪽 NN관심있어서 논문읽고 공부하며 kaist 랩에서 인턴으로 참여중에 있다.
9
곰은 전 세계 사람들의 마음과 상상력을 사로잡은 강력하고 장엄한 생물이지만 많은 곰 종은 멸종 위기에 처하거나 취약한 것으로 분류되어 개체수가 감소하고 있다.
10
데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 데이터보내는용 앱 대학원에서 계량심

### 중요 문장 추출

### 중요 단어 추출

In [26]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install sentence_transformers
!pip install konlpy

In [28]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
text_data = pd.read_csv('/content/drive/MyDrive/chat_test.csv')
text_data.head(5)

In [ ]:
okt = Okt()
# total_words = []

for index, row in text_data.iterrows():
    doc = row['summary']
    tokenized_doc = okt.pos(doc)
    # tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun' or word[1]=='Alpha') and len(word[1])>2])

    nouns = [word[0] for word in tokenized_doc if (word[1] == 'Noun' or word[1] == 'Alpha') and len(word[1]) > 2]

    #print('품 사 태 깅 10개 만 출 력 :',tokenized_doc[:10])
    unique_words = list(set(nouns))
    word_list = []
    for line in unique_words:
        if len(line) > 2:
            word_list.append(line)
    print('명 사 추 출 :', word_list)
    # total_words.extend(word_list)

TF-IDF 이용(진유)

Textrank 이용(은주)

### notion database로 옮김

In [ ]:
!pip install python-dotenv # (※ 주의 : pip install dotenv가 아님)
!pip install notion-client

In [65]:
!dotenv set NOTION_TOKEN secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV
!dotenv list

NOTION_TOKEN=secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV
NOTION_TOKEN=secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV


In [66]:
from dotenv import dotenv_values
from notion_client import Client

config = dotenv_values(".env")
notion_secret = config.get('NOTION_TOKEN')
notion = Client(auth=notion_secret)

In [69]:
import os

from google.colab import drive
drive.mount('/content/drive')

import requests
from pprint import pprint

notion_api_key = "secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV"
database_id = "79c35ba564e0447aa7165c4f51795eae"

headers = {
    "Authorization": f"Bearer {notion_api_key}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

url = f"https://api.notion.com/v1/pages"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [89]:
text_data = pd.read_csv('/content/drive/MyDrive/chat_test.csv')
count = 0

for index, row in text_data.iterrows():
  count+=1
  doc = row['summary']
  text = row['text']
  tokenized_doc = okt.pos(str(doc))
 
  nouns = [word[0] for word in tokenized_doc if (word[1] == 'Noun' or word[1] == 'Alpha') and len(word[1]) > 2]

  unique_words = list(set(nouns))
  word_list = []
  for line in unique_words:
      if len(line) > 2 and line != 'nan':
          word_list.append(line)
  print('명사추출 :', word_list)

  tag_list = [ ]
  for word in word_list:
      tag_list.append({"name": word})

  data = {
        "parent": {"database_id": database_id},
        "properties": {
            "Name": {
                "title": [
                    {
                        "text": {
                            "content": doc
                        }
                    }
                ]
            },
            # Add a multi-select property for tags
            "Tags": {
                "multi_select": tag_list
            },
            "Text": {
                "rich_text": [
                    {
                        "text": {
                            "content": text
                        }
                    }
                ]
            }
        }
    }

  response = requests.post(url, headers=headers, json=data)
  #result = response.json()
  #pprint(result)

  if count >10:
    break

명사추출 : ['캐나다', 'OpenAI', 'microsoft', '저작권']
명사추출 : ['데이터', '스크립트', '드라마', 'GraphViz', '그래프', 'ChatGPT']
명사추출 : ['데이터', '사용자', '비공개', '상품화', '연구소', 'ChatGPT']
명사추출 : []
명사추출 : []
명사추출 : []
명사추출 : ['다빈치']
명사추출 : ['글쓰기']
명사추출 : ['graph', '서울대학교', 'kaist']
명사추출 : ['상상력']
명사추출 : ['데이터', '대학원']


1. database의 주소에서 ?v 앞이 database 주소
2. integration을 해주어야 함. -> add connection. 미리 등록한 integration 등록하기. : https://developers.notion.com/docs/create-a-notion-integration
